In [1]:
from dumbnano import NanoAmpliParser

In [2]:
#Initialize of the parser, seems to be meaningless, but it is not.
#In OOP, initialization is a must.
dumb = NanoAmpliParser()

In [ ]:
#combine_fastq will search all fastq.gz in src folder and save them into one single fastq in des folder
#This method will return the path of the combined fastq file
all_fastq = dumb.combine_fastq(src = "./reads/bonito/", 
                               des = "./reads/1_onefastq/"
                               )

In [ ]:
#nanoflit read the fastq file and filter out low quality reads, save the filtered reads into des folder
#this method return the path of the filtered fastq file
fastq_nano = dumb.nanoflit(src = './reads/1_onefastq/all.fastq',
                           des = './reads/2_nanoflit/' ,
                           NANOFILT_QSCORE = 8,  #recommended 7-9
                           NANOFILT_MIN_LEN = 800, #depends on the length of your reads 
                           NANOFILT_MAX_LEN = 7000 #depends on the length of your reads
                           )

In [3]:
#Singlebar take fastq file as input and demultiplex the reads into different files according to the barcodes
#Barcode info should be provided in a tsv file. 
#Barcode info should have three columns: SampleID, FwIndex and RvAnchor
#SampleID is the name of the sample
#FwIndex is the barcode sequence that is located at the beginning of the read
#RvAnchor is the sequence that is located at the end of the read, it is used to confirm the completeness of the read
#mismatch_ratio_f and mismatch_ratio_r are the mismatch ratio allowed for the barcode and the anchor sequence
#recommended mismatch ratio is 0.1, which means given a 20bp barcode, 2bp mismatch is allowed
demultiplexed = dumb.singlebar(src = './reads/2_nanoflit/all_nano.fastq',
                               des = './reads/3_demultiplexed/', 
                               BARCODE_INDEX_FILE="./reads/2023000003_barcode.tsv - 230107_barcode.tsv", 
                               mismatch_ratio_f = 0.1, 
                               mismatch_ratio_r = 0.1
                               )

/home/raingel/桌面/OOP_v2/dumbnano.py:206: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  BARCODE_IDX_DF = pd.read_csv(BARCODE_INDEX_FILE, sep="\\t")


10000
20000
30000
40000
50000
60000
15140/60488 (25.03%) reads were demultiplexed successfully


In [3]:
#This method is used to remove the artificial sequence at the beginning and the end of the read
#minibar or singlebar will mark the position of the barcode and the anchor sequence in lower case
#Example:  TACTTCGTTCAGTTACGTATTGCTagttcgttaggcggctgatTGCTATGCGCGAGCTGCAAGTTTGATTATGGCTCAGGGAG......GGATGCCGAAGGCAGGGCTAGTGACTGGGTGAAGTCGTAACAAGGTAACCGTAgcagctcgcgcatagCA
#deHead will remove the sequence before the second uppercase letter and after the second last uppercase letter
#Start_offset and end_offset are used to adjust the position of cut off point
#Example with start_offset = 10 and end_offset = 5, the result will be:
#TACTTCGTTCAGTTACGTATTGCTagttcgttaggcggctgatTGCTATGCGC//GAGCTGCAAGTTTGATTATGGCTCAGGGAG......GGATGCCGAAGGCAGGGCTAGTGACTGGGTGAAGTCGTAACAAGGTAA//CCGTAgcagctcgcgcatagCA
#                                                       GAGCTGCAAGTTTGATTATGGCTCAGGGAG......GGATGCCGAAGGCAGGGCTAGTGACTGGGTGAAGTCGTAACAAGGTAA
deHead = dumb.deHead(src='./reads/3_demultiplexed/',
                     des='./reads/4_deHead',
                     start_offset=40,
                    end_offset=20)

In [4]:
# HDBSCAN is a density-based clustering algorithm that automatically
# determines the number of clusters and identifies noise in the data.
# The `min_cluster_size` parameter specifies the minimum number of points
# required for a cluster to be formed. Any cluster with fewer points than
# this threshold will be considered noise and labeled as such.

# Setting the `min_cluster_size` parameter to a small value will allow
# the algorithm to detect smaller clusters in the data, but it may also
# result in more noise being included as clusters. On the other hand,
# setting the `min_cluster_size` parameter to a large value will result
# in fewer clusters being detected but with more confidence, as noise
# and smaller clusters will be discarded.

# It is important to choose an appropriate value for `min_cluster_size`
# based on the characteristics of the data and the specific problem you
# are trying to solve. 

#In following code, min_cluster_size represents the ratio of the minimum number of reads in a cluster to the total number of reads
#default is 0.2, which means the minimum number of reads in a cluster is 0.2*total number of reads

cluster = dumb.clusters_fastas(src = './reads/4_deHead/',
                               des = './reads/5_clustered/',
                               min_cluster_size = 0.1, #recommended 0.1-0.3
                               mds=True #Produce a MDS plot to visualize the clustering result
                               )

Clustering 2110.fas
Number of records: 32
abs_cluster_size:  3
Number of clusters: 6
Clustering 2279.fas
Number of records: 90
abs_cluster_size:  9
Number of clusters: 1
Clustering 2131.fas
Number of records: 1078
abs_cluster_size:  107
Number of clusters: 3


KeyboardInterrupt: 